## Supplement: Choosing aperture size

Fukui+2016a:
```
After several trials with different aperture radii, we select the optimal aperture radius such that the resultant relative light curve produced by dividing the target-star flux by the sum of the fluxes of the two comparison stars gives the minimum dispersion relative to the best-fitted transit model.
```

In [1]:
#check python version
!python --version

Python 3.6.1 :: Continuum Analytics, Inc.


In [2]:
#check environment
import os
os.environ['CONDA_DEFAULT_ENV']

'astroconda35'

## import data

In [3]:
import glob

dirname='../HAT-P-44b'
fname = os.path.join(dirname,'*.dat')
file_list = glob.glob(fname)
file_list

['../HAT-P-44b/lcf_msct_z_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_r_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_z_hatp44_170215_t2_c1_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_r_hatp44_170215_t2_c1_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_z_hatp44_170215_t2_c1_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_r_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_z_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_r_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_g_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_g_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_z_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_g_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_r_hatp44_170215_t2_c13_r9-14.bjd.dat',
 '../HAT-P-44b/lcf_msct_z_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_g_hatp44_170215_t2_c1_r9-14.bjd.dat',
 '../HAT-P-44b/lcm_msct_g_hatp44_170215_t2_c3_r9-14.bjd.dat',
 '

## Target, Comparison ID: 2, 1

In [4]:
name               = 'hatp44'
date               = '170215'
target_star_id     = '2'
comparison_star_id = '1'
radii_range        = '9-14'
b                  = 'g'

#filename
fname='lcf_msct_'+b+'_'+name+'_'+date+'_t'+target_star_id+'_c'+comparison_star_id+'_r'+radii_range+'.bjd.dat'
path = os.path.join(dirname,fname)
path

'../HAT-P-44b/lcf_msct_g_hatp44_170215_t2_c1_r9-14.bjd.dat'

In [5]:
import pandas as pd
import numpy as np

data = {}

bands   = ['g','r','z']

sigma = 5.0


for b in bands:
    fname = 'lcf_msct_'+b+'_'+name+'_'+date+'_t'+target_star_id+'_c'+\
            comparison_star_id+'_r'+radii_range+'.bjd.dat'
    
    #combine directory name and filename above
    path  = os.path.join(dirname,fname)
    
    #read data and save into a dataframe
    df=pd.read_csv(path, delimiter=' ', parse_dates=True)
    
    #remove columns (axis=1) with names specified
    df=df.drop(['Unnamed: 20','frame'],axis=1)
    
    #save the dataframe with key "b" into a dictionary named "data"
    df=df.set_index('BJD(TDB)-2450000')
    
    #find outliers
    mask = np.abs(df-df.mean())>(sigma*df.std())
    
    #remove outliers
    df = df[~mask]
    
    #remove nan
    df = df.dropna()
    
    data[b] =df

In [6]:
import numpy as np
from astropy import units as u

_tc  = 2455696.93695
_P   = 4.301219
_inc = np.deg2rad(89.10)
_t14  = 0.13020
_b    = 0.172
_k    = np.sqrt(0.01804) #± 0.np.sqrt(00027)
Rp = 1.24 #Rjup
Rs = 0.949*u.Rsun.to(u.Rjup) #Rsol to Rjup
k_ = Rp/Rs

_a_s = 11.52
#a_s_  = scaled_a(_P, _t14, k_, i=_inc, impact_param=_b)

tc_0      = 7.8e3+0.22 #-2450000

In [7]:
def u_to_q(u1, u2):
    '''
    convert limb-darkening coefficients
    from u to q
    
    see Eq. 15 & 16 in Kipping 2013
    '''
    q1 = (u1 + u2)**2
    q2 = u1 / (2 * (u1 + u2))
    return q1, q2

def q_to_u(q1, q2):
    '''
    convert limb-darkening coefficients
    from q to u
    
    see Eq. 17 & 18 in Kipping 2013
    '''
    u1 = 2 * np.sqrt(q1) * q2
    u2 = np.sqrt(q1) * (1 - 2*q2)
    return u1, u2

In [8]:
from pytransit import MandelAgol

def transit_model_q(parameters, period, time, model=MandelAgol()):
    '''
    Compute flux using the Mandel-Agol model:
    \frac{I(\mu)}{I(1)} = 1 − u_1(1 − \mu) − u_2(1 − \mu)^2
    '''
    k,q1,q2,tc,a,b = parameters
    
    #compute inclination
    inc   = np.arccos(b/a)
    #convert u to q
    u1,u2 = q_to_u(q1, q2)
    #evaluate the model
    m = model.evaluate(time, k, (u1,u2), tc, period, a, inc)
    
    return m

See part5 for log likelihood of a transit model.

In [9]:
def loglike(params, p, t, f, err):
    '''
    log likelihood of the transit model
    '''
    assert len(params) == 6
    m = transit_model_q(params, p, t)
        
    resid = f - m
    
    #-0.5*(np.sum((resid)**2 * np.exp(-2*ls) + 2*ls))
    C = np.log(2*np.pi)
    
    return -0.5*(np.sum(np.log(err) + C + (resid/err)**2))

Alternatively, use the optimized transit parameters from part5.

In [10]:
optimized_transit_params= {'g': np.array([  1.28330953e-01,   6.94722181e-01,   4.65491536e-01,
          7.80021976e+03,   1.21625800e+01,   1.72028975e-01]),
 'r': np.array([  1.28408329e-01,   5.53907376e-01,   3.49739851e-01,
          7.80021952e+03,   1.22467056e+01,   1.72044914e-01]),
 'z': np.array([  1.25171086e-01,   3.43090243e-01,   2.07678538e-01,
          7.80021956e+03,   1.22930550e+01,   1.71164788e-01])}

In [11]:
df.columns

Index(['airmass', 'sky(ADU)', 'dx(pix)', 'dy(pix)', 'fwhm(pix)', 'peak(ADU)',
       'flux(r=9.0)', 'err(r=9.0)', 'flux(r=10.0)', 'err(r=10.0)',
       'flux(r=11.0)', 'err(r=11.0)', 'flux(r=12.0)', 'err(r=12.0)',
       'flux(r=13.0)', 'err(r=13.0)', 'flux(r=14.0)', 'err(r=14.0)'],
      dtype='object')

In [30]:
def rms(flux,flux_model):
    residual = flux-flux_model
    #return np.sqrt((residual**2).sum()/residual.size)
    
    #(flux/sys_model - transit_model) / error,
    #or (flux - transit_model*sys_model) / error

    return np.sqrt(np.mean((residual)**2))

In [14]:
def chisq(resid, sig, ndata=None, nparams=None, reduced=False):
    if reduced:
        assert ndata is not None and nparams is not None
        dof = ndata - nparams
        return sum((resid / sig)**2) / (dof)
    else:
        return sum((resid / sig)**2)

In [36]:
bands = 'g,r,z'.split(',')


RMS       = {}

print('aperture\trms (%)')
for b in bands:
    df = data[b]
    
    apertures = {}
    
    print('{}-band'.format(b))
    for i in np.arange(9,15,1):
        aperture = '(r={:.1f})'.format(i)
        errcol  = 'err' + aperture
        fluxcol = 'flux'+ aperture
                
        time = df.index
        flux = df[fluxcol]
        err  = df[errcol]
        #q1_,q2_ = u_to_q(ldc_list[0],ldc_list[1])
        #params = [_k,q1_,q2_,tc_0,_a_s,_b]
        #transit_model_before  = transit_model_q(transit_params, _P, time)

        #rms_before = rms(flux,transit_model_before)
        #print('rms before: {:.4f}'.format(rms_before))

        #optimize parameters
        #result = op.minimize(obj, transit_params,
        #                     args=(_P, time, flux, err), method='nelder-mead')
        params = optimized_transit_params[b]
        transit_model = transit_model_q(params, _P, time)
        
        rms_err = rms(flux,transit_model)*100 #in %
        
        print('{}:\t{:.3f}'.format(aperture, rms_err))
        apertures[i] = rms_err
        
    RMS[b] = apertures

aperture	rms (%)
g-band
(r=9.0):	0.297
(r=10.0):	0.268
(r=11.0):	0.264
(r=12.0):	0.263
(r=13.0):	0.269
(r=14.0):	0.273
r-band
(r=9.0):	0.281
(r=10.0):	0.267
(r=11.0):	0.260
(r=12.0):	0.265
(r=13.0):	0.259
(r=14.0):	0.259
z-band
(r=9.0):	0.268
(r=10.0):	0.271
(r=11.0):	0.280
(r=12.0):	0.279
(r=13.0):	0.284
(r=14.0):	0.290


In [37]:
apers = list(apertures.keys())

print('aperture with minimum rms error (%):')
for ap,b in zip(apers,bands):
    print('{}-band'.format(b))
    
    idx=np.argmin(RMS[b].values())
    key = apers[idx]
    min_rms = RMS[b][key]
    print('r={}.0:\trms:{:.2f}'.format(key,min_rms))

aperture with minimum rms error (%):
g-band
r=9.0:	rms:0.30
r-band
r=9.0:	rms:0.28
z-band
r=9.0:	rms:0.27


## Target, Comparison ID: 2, 3

In [76]:
name               = 'hatp44'
date               = '170215'
target_star_id     = '2'
comparison_star_id = '3'
radii_range        = '9-14'
b                  = 'g'

#filename
fname='lcf_msct_'+b+'_'+name+'_'+date+'_t'+target_star_id+'_c'+comparison_star_id+'_r'+radii_range+'.bjd.dat'
path = os.path.join(dirname,fname)
path

'../HAT-P-44b/lcf_msct_g_hatp44_170215_t2_c3_r9-14.bjd.dat'

In [77]:
data2 = {}

sigma = 5.0


for b in bands:
    fname = 'lcf_msct_'+b+'_'+name+'_'+date+'_t'+target_star_id+'_c'+\
            comparison_star_id+'_r'+radii_range+'.bjd.dat'
    
    #combine directory name and filename above
    path  = os.path.join(dirname,fname)
    
    #read data and save into a dataframe
    df=pd.read_csv(path, delimiter=',', parse_dates=True)
    
    #remove columns (axis=1) with names specified
    df = pd.read_csv(path, delimiter=' ', parse_dates=True)
    cols = np.array(df.columns.delete(0))
    cols = np.append(cols,'dummy')

    df.columns = cols
    df = df.drop(['Unnamed: 21','frame','dummy'],axis=1)
    df = df.set_index('BJD(TDB)-2450000')
    
    #find outliers
    mask = np.abs(df-df.mean())>(sigma*df.std())
    
    #remove outliers
    df = df[~mask]
    
    #remove nan
    df = df.dropna()
    
    data2[b] =df

In [78]:
bands = 'g,r,z'.split(',')


RMS2       = {}

print('aperture\trms (%)')
for b in bands:
    df = data2[b]
    
    apertures = {}
    
    print('{}-band'.format(b))
    for i in np.arange(9,15,1):
        aperture = '(r={:.1f})'.format(i)
        errcol  = 'err' + aperture
        fluxcol = 'flux'+ aperture
                
        time = df.index
        flux = df[fluxcol]
        err  = df[errcol]
        #q1_,q2_ = u_to_q(ldc_list[0],ldc_list[1])
        #params = [_k,q1_,q2_,tc_0,_a_s,_b]
        #transit_model_before  = transit_model_q(transit_params, _P, time)

        #rms_before = rms(flux,transit_model_before)
        #print('rms before: {:.4f}'.format(rms_before))

        #optimize parameters
        #result = op.minimize(obj, transit_params,
        #                     args=(_P, time, flux, err), method='nelder-mead')
        params = optimized_transit_params[b]
        transit_model = transit_model_q(params, _P, time)
        
        rms_err = rms(flux,transit_model)*100 #in %
        
        print('{}:\t{:.3f}'.format(aperture, rms_err))
        apertures[i] = rms_err
        
    RMS2[b] = apertures

aperture	rms (%)
g-band
(r=9.0):	0.398
(r=10.0):	0.404
(r=11.0):	0.405
(r=12.0):	0.423
(r=13.0):	0.449
(r=14.0):	0.454
r-band
(r=9.0):	0.315
(r=10.0):	0.307
(r=11.0):	0.319
(r=12.0):	0.327
(r=13.0):	0.346
(r=14.0):	0.353
z-band
(r=9.0):	0.444
(r=10.0):	0.410
(r=11.0):	0.387
(r=12.0):	0.395
(r=13.0):	0.394
(r=14.0):	0.397


In [79]:
apers = list(apertures.keys())

print('aperture with minimum rms error (%):')
for ap,b in zip(apers,bands):
    print('{}-band'.format(b))
    
    idx=np.argmin(RMS[b].values())
    key = apers[idx]
    min_rms = RMS2[b][key]
    print('r={}.0:\trms:{:.2f}'.format(key,min_rms))

aperture with minimum rms error (%):
g-band
r=9.0:	rms:0.40
r-band
r=9.0:	rms:0.31
z-band
r=9.0:	rms:0.44
